## Import library

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pickle
import cv2
from tqdm import tqdm
from pathlib import Path
import imageio as iio
import imgaug.augmenters as ia
import imgaug
import numpy as np
from PIL import Image, ImageOps

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import random_split
import matplotlib.pyplot as plt
import time
import os
import copy

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),

}

data_dir = '/content/drive/MyDrive/Xu ly anh /Group_Thanh_Ha/datadir'

class_names = sorted(os.listdir(data_dir))

# 100 images - trial 1

In [ ]:
# Load the model
loaded_model = torch.load('/content/drive/MyDrive/cs406/project/TRECVID/checkpoints/best_model_aug-resnet_100_1.pth')

# If you want to use the loaded model for inference, make sure to set it to evaluation mode
loaded_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
test_folder = '/content/drive/MyDrive/cs406/project/TRECVID/test_set'
count = 0
count_true = 0
for class_name in os.listdir(test_folder):
  count_true_class = 0
  count_class = 0
  for image_name in os.listdir(os.path.join(test_folder, class_name)):
      image_test = os.path.join(test_folder, class_name, image_name)
      image = Image.open(image_test)
      transform = data_transforms['valid']  # Sử dụng chuyển đổi cho tập kiểm thử
      input_image = transform(image)
      input_image = input_image.unsqueeze(0)
      input_image = input_image.to(device)
      with torch.no_grad():
          outputs = loaded_model(input_image)
      _, preds = torch.max(outputs, 1)
      true_class = image_name[:-6]
      if(true_class == class_names[preds.item()]):
        count_true_class+=1
      count_class += 1
  print(f'Movie {class_name}: {count_true_class}/{count_class}')
  count += count_class
  count_true += count_true_class

Movie like_me: 16/20
Movie liberty_kid: 16/26
Movie little_rock: 17/23
Movie chained_for_life: 10/17
Movie losing_ground: 21/33
Movie calloused_hands: 28/40


In [ ]:
print(f'Total: {count_true}/{count}')
print(f'Acc: {count_true/count*100:.2f}')

Total: 108/159
Acc: 67.92


# 100 images - trial 2

In [ ]:
# Load the model
loaded_model = torch.load('/content/drive/MyDrive/cs406/project/TRECVID/checkpoints/best_model_aug-resnet_100_2.pth')

# If you want to use the loaded model for inference, make sure to set it to evaluation mode
loaded_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
test_folder = '/content/drive/MyDrive/cs406/project/TRECVID/test_set'
count = 0
count_true = 0
for class_name in os.listdir(test_folder):
  count_true_class = 0
  count_class = 0
  for image_name in os.listdir(os.path.join(test_folder, class_name)):
      image_test = os.path.join(test_folder, class_name, image_name)
      image = Image.open(image_test)
      transform = data_transforms['valid']  # Sử dụng chuyển đổi cho tập kiểm thử
      input_image = transform(image)
      input_image = input_image.unsqueeze(0)
      input_image = input_image.to(device)
      with torch.no_grad():
          outputs = loaded_model(input_image)
      _, preds = torch.max(outputs, 1)
      true_class = image_name[:-6]
      if(true_class == class_names[preds.item()]):
        count_true_class+=1
      count_class += 1
  print(f'Movie {class_name}: {count_true_class}/{count_class}')
  count += count_class
  count_true += count_true_class

Movie like_me: 16/20
Movie liberty_kid: 16/26
Movie little_rock: 16/23
Movie chained_for_life: 10/17
Movie losing_ground: 21/33
Movie calloused_hands: 25/40


In [ ]:
print(f'Total: {count_true}/{count}')
print(f'Acc: {count_true/count*100:.2f}')

Total: 104/159
Acc: 65.41


# 200 images trial 1

In [ ]:
# Load the model
loaded_model = torch.load('/content/drive/MyDrive/cs406/project/TRECVID/checkpoints/best_model_aug-resnet_200_1.pth')

# If you want to use the loaded model for inference, make sure to set it to evaluation mode
loaded_model.eval()

test_folder = '/content/drive/MyDrive/cs406/project/TRECVID/test_set'
count = 0
count_true = 0
for class_name in os.listdir(test_folder):
  count_true_class = 0
  count_class = 0
  for image_name in os.listdir(os.path.join(test_folder, class_name)):
      image_test = os.path.join(test_folder, class_name, image_name)
      image = Image.open(image_test)
      transform = data_transforms['valid']  # Sử dụng chuyển đổi cho tập kiểm thử
      input_image = transform(image)
      input_image = input_image.unsqueeze(0)
      input_image = input_image.to(device)
      with torch.no_grad():
          outputs = loaded_model(input_image)
      _, preds = torch.max(outputs, 1)
      true_class = image_name[:-6]
      if(true_class == class_names[preds.item()]):
        count_true_class+=1
      count_class += 1
  print(f'Movie {class_name}: {count_true_class}/{count_class}')
  count += count_class
  count_true += count_true_class

print(f'Total: {count_true}/{count}')
print(f'Acc: {count_true/count*100:.2f}')

Movie like_me: 15/20
Movie liberty_kid: 17/26
Movie little_rock: 17/23
Movie chained_for_life: 10/17
Movie losing_ground: 24/33
Movie calloused_hands: 28/40
Total: 111/159
Acc: 69.81


# 200 images trial 2

In [ ]:
# Load the model
loaded_model = torch.load('/content/drive/MyDrive/cs406/project/TRECVID/checkpoints/best_model_aug-resnet_200_2.pth')

# If you want to use the loaded model for inference, make sure to set it to evaluation mode
loaded_model.eval()

test_folder = '/content/drive/MyDrive/cs406/project/TRECVID/test_set'
count = 0
count_true = 0
for class_name in os.listdir(test_folder):
  count_true_class = 0
  count_class = 0
  for image_name in os.listdir(os.path.join(test_folder, class_name)):
      image_test = os.path.join(test_folder, class_name, image_name)
      image = Image.open(image_test)
      transform = data_transforms['valid']  # Sử dụng chuyển đổi cho tập kiểm thử
      input_image = transform(image)
      input_image = input_image.unsqueeze(0)
      input_image = input_image.to(device)
      with torch.no_grad():
          outputs = loaded_model(input_image)
      _, preds = torch.max(outputs, 1)
      true_class = image_name[:-6]
      if(true_class == class_names[preds.item()]):
        count_true_class+=1
      count_class += 1
  print(f'Movie {class_name}: {count_true_class}/{count_class}')
  count += count_class
  count_true += count_true_class

print(f'Total: {count_true}/{count}')
print(f'Acc: {count_true/count*100:.2f}')

Movie like_me: 17/20
Movie liberty_kid: 15/26
Movie little_rock: 17/23
Movie chained_for_life: 9/17
Movie losing_ground: 22/33
Movie calloused_hands: 27/40
Total: 107/159
Acc: 67.30


# 500 images trial 1

In [ ]:
# Load the model
loaded_model = torch.load('/content/drive/MyDrive/cs406/project/TRECVID/checkpoints/best_model_aug-resnet_500_1.pth')

# If you want to use the loaded model for inference, make sure to set it to evaluation mode
loaded_model.eval()

test_folder = '/content/drive/MyDrive/cs406/project/TRECVID/test_set'
count = 0
count_true = 0
for class_name in os.listdir(test_folder):
  count_true_class = 0
  count_class = 0
  for image_name in os.listdir(os.path.join(test_folder, class_name)):
      image_test = os.path.join(test_folder, class_name, image_name)
      image = Image.open(image_test)
      transform = data_transforms['valid']  # Sử dụng chuyển đổi cho tập kiểm thử
      input_image = transform(image)
      input_image = input_image.unsqueeze(0)
      input_image = input_image.to(device)
      with torch.no_grad():
          outputs = loaded_model(input_image)
      _, preds = torch.max(outputs, 1)
      true_class = image_name[:-6]
      if(true_class == class_names[preds.item()]):
        count_true_class+=1
      count_class += 1
  print(f'Movie {class_name}: {count_true_class}/{count_class}')
  count += count_class
  count_true += count_true_class

print(f'Total: {count_true}/{count}')
print(f'Acc: {count_true/count*100:.2f}')

Movie like_me: 17/20
Movie liberty_kid: 15/26
Movie little_rock: 17/23
Movie chained_for_life: 9/17
Movie losing_ground: 22/33
Movie calloused_hands: 28/40
Total: 108/159
Acc: 67.92


# 500 images trial 2

In [ ]:
# Load the model
loaded_model = torch.load('/content/drive/MyDrive/cs406/project/TRECVID/checkpoints/best_model_aug-resnet_500_1.pth')

# If you want to use the loaded model for inference, make sure to set it to evaluation mode
loaded_model.eval()

test_folder = '/content/drive/MyDrive/cs406/project/TRECVID/test_set'
count = 0
count_true = 0
for class_name in os.listdir(test_folder):
  count_true_class = 0
  count_class = 0
  for image_name in os.listdir(os.path.join(test_folder, class_name)):
      image_test = os.path.join(test_folder, class_name, image_name)
      image = Image.open(image_test)
      transform = data_transforms['valid']  # Sử dụng chuyển đổi cho tập kiểm thử
      input_image = transform(image)
      input_image = input_image.unsqueeze(0)
      input_image = input_image.to(device)
      with torch.no_grad():
          outputs = loaded_model(input_image)
      _, preds = torch.max(outputs, 1)
      true_class = image_name[:-6]
      if(true_class == class_names[preds.item()]):
        count_true_class+=1
      count_class += 1
  print(f'Movie {class_name}: {count_true_class}/{count_class}')
  count += count_class
  count_true += count_true_class

print(f'Total: {count_true}/{count}')
print(f'Acc: {count_true/count*100:.2f}')

Movie like_me: 17/20
Movie liberty_kid: 15/26
Movie little_rock: 17/23
Movie chained_for_life: 9/17
Movie losing_ground: 22/33
Movie calloused_hands: 28/40
Total: 108/159
Acc: 67.92


# Without augmentation

In [ ]:
# Load the model
loaded_model = torch.load('/content/drive/MyDrive/cs406/project/TRECVID/checkpoints/model_resnet_without_augmentation.pth')

# If you want to use the loaded model for inference, make sure to set it to evaluation mode
loaded_model.eval()

test_folder = '/content/drive/MyDrive/cs406/project/TRECVID/test_set'
count = 0
count_true = 0
for class_name in os.listdir(test_folder):
  count_true_class = 0
  count_class = 0
  for image_name in os.listdir(os.path.join(test_folder, class_name)):
      image_test = os.path.join(test_folder, class_name, image_name)
      image = Image.open(image_test)
      transform = data_transforms['valid']  # Sử dụng chuyển đổi cho tập kiểm thử
      input_image = transform(image)
      input_image = input_image.unsqueeze(0)
      input_image = input_image.to(device)
      with torch.no_grad():
          outputs = loaded_model(input_image)
      _, preds = torch.max(outputs, 1)
      true_class = image_name[:-6]
      if(true_class == class_names[preds.item()]):
        count_true_class+=1
      count_class += 1
  print(f'Movie {class_name}: {count_true_class}/{count_class}')
  count += count_class
  count_true += count_true_class

print(f'Total: {count_true}/{count}')
print(f'Acc: {count_true/count*100:.2f}')

Movie like_me: 0/20
Movie liberty_kid: 2/26
Movie little_rock: 1/23
Movie chained_for_life: 3/17
Movie losing_ground: 7/33
Movie calloused_hands: 4/40
Total: 17/159
Acc: 10.69


# Channel shuffle

In [ ]:
# Load the model
loaded_model = torch.load('/content/drive/MyDrive/cs406/project/TRECVID/checkpoints/model_resnet_channel_shuffle.pth')

# If you want to use the loaded model for inference, make sure to set it to evaluation mode
loaded_model.eval()

test_folder = '/content/drive/MyDrive/cs406/project/TRECVID/test_set'
count = 0
count_true = 0
for class_name in os.listdir(test_folder):
  count_true_class = 0
  count_class = 0
  for image_name in os.listdir(os.path.join(test_folder, class_name)):
      image_test = os.path.join(test_folder, class_name, image_name)
      image = Image.open(image_test)
      transform = data_transforms['valid']  # Sử dụng chuyển đổi cho tập kiểm thử
      input_image = transform(image)
      input_image = input_image.unsqueeze(0)
      input_image = input_image.to(device)
      with torch.no_grad():
          outputs = loaded_model(input_image)
      _, preds = torch.max(outputs, 1)
      true_class = image_name[:-6]
      if(true_class == class_names[preds.item()]):
        count_true_class+=1
      count_class += 1
  print(f'Movie {class_name}: {count_true_class}/{count_class}')
  count += count_class
  count_true += count_true_class

print(f'Total: {count_true}/{count}')
print(f'Acc: {count_true/count*100:.2f}')

Movie like_me: 2/20
Movie liberty_kid: 6/26
Movie little_rock: 5/23
Movie chained_for_life: 5/17
Movie losing_ground: 3/33
Movie calloused_hands: 8/40
Total: 29/159
Acc: 18.24
